# Exploraty Data Analysis

Exploraty Data Analysis (EDA) of our datasets about the Drupal Community.

## Installation

In [29]:
import datetime
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from pathlib import Path
import shutil
import requests
import zipfile

# Define paths
download_dir = Path.home()
zip_file_path = download_dir / "country-codes-iso-3166.zip"
extract_dir = Path.cwd() / "../data/csv"

# Ensure the extract directory exists
extract_dir.mkdir(parents=True, exist_ok=True)

# Download the dataset
url = "https://www.kaggle.com/api/v1/datasets/download/wbdill/country-codes-iso-3166"
response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(zip_file_path, "wb") as f:
        shutil.copyfileobj(response.raw, f)
    print(f"Downloaded to {zip_file_path}")
else:
    print(f"Failed to download. Status code: {response.status_code}")

# Unzip the file
if zip_file_path.exists():
    with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Extracted to {extract_dir}")

# Remove the zip file
if zip_file_path.exists():
    zip_file_path.unlink()
    print(f"Removed zip file: {zip_file_path}")

Downloaded to /home/ubuntu/country-codes-iso-3166.zip
Extracted to /home/ubuntu/projects/drucom/notebook/../data/csv
Removed zip file: /home/ubuntu/country-codes-iso-3166.zip


## Users

### Load the dataset

The source `user.json` file has been compiled with `../script/fetch_drupal_data.sh`.

It is an extract of all the records exposed by the following Drupal.org REST API endpoint: https://www.drupal.org/api-d7/user.json?sort=uid&direction.

It does not contain all the existing data but a simplified/minimalistic set.

In [ ]:

# Load user data - should take ~2min.
dtypes = {
    "id": "int32",
    "title": "string",
    "fname": "string",
    "lname": "string",
    "created": "int32",
    "da_membership": "string",
    "slack": "string",
    "timezone": "string",
    "region": "string",
    "mentors": "object",
    "countries": "object",
    "languages": "object",
    "organizations": "object",
    "industries": "object",
    "contributions": "object",
    "events": "object",
}

users = pd.read_json('../data/json/user.json', dtype=dtypes)

In [27]:
# Add formatted registration date.
if 'registered_on' not in users.columns:
    users['registered_on'] = users['created'].apply(lambda d: datetime.datetime.fromtimestamp(d))

In [ ]:
# Clean empty arrays.
for col in users.columns:
    if users[col].dtype == 'O':
        users[col] = users[col].apply(lambda x: None if (x is None or len(x) == 0) else x)

In [ ]:
# Save cleaned data.
users.to_csv("../data/csv/user_data.csv", index=False)

### Overview

Analyse of the `user.json` cleaned data file. 

It contains approximately 2.1 million lines as of March 23rd, 2025.

In [9]:
# Shape, columns, and sample of the data.
display(users.shape, users.columns, users.sample(5))

(2093461, 19)

Index(['id', 'title', 'fname', 'lname', 'created', 'da_membership', 'slack',
       'mentors', 'countries', 'language', 'languages', 'timezone', 'region',
       'city', 'organizations', 'industries', 'contributions', 'events',
       'registered_on'],
      dtype='object')

,id,title,fname,lname,created,da_membership,slack,mentors,countries,language,languages,timezone,region,city,organizations,industries,contributions,events,registered_on
136417,139322,admin@test.openfish.info,<NA>,<NA>,1177778629,<NA>,<NA>,None,None,None,None,<NA>,<NA>,None,None,None,None,None,2007-04-28 18:43:49
559641,716830,kacenka1524,<NA>,<NA>,1265620626,<NA>,<NA>,None,[SK],None,None,<NA>,<NA>,None,None,None,None,None,2010-02-08 10:17:06
1522197,2823727,tarlan1994,<NA>,<NA>,1392853989,<NA>,<NA>,None,None,None,None,UTC,UTC,UTC,None,None,None,None,2014-02-20 00:53:09
1272449,2305520,zodiacZ,Zodiac,Zone,1348309148,<NA>,<NA>,None,[US],None,None,<NA>,<NA>,None,None,None,None,None,2012-09-22 12:19:08
1242440,2232928,benedict123,grant,vargas,1344082496,<NA>,<NA>,None,[PH],None,None,<NA>,<NA>,None,None,None,None,None,2012-08-04 14:14:56


In [ ]:
display("Empty values:", users.isnull().sum())

'Empty values:'

id                     0
title                  0
fname            1609121
lname            1620031
created                0
da_membership    2090786
slack            2090083
mentors          2086534
countries         807087
language         2080505
languages        1986375
timezone         1425524
region           1426210
city             1426210
organizations    1824515
industries       2093461
contributions    2064714
events           2079544
registered_on          0
dtype: int64

In [53]:
# Display count of unique values for interesting columns.
for col in ['da_membership', 'language', 'city', 'region']:
    display(f"Column: {col}", users[col].value_counts())

# Display total of each array field with a list of values.
from collections import Counter

totals = {}
for col in users.columns:
    if users[col].dtype == 'O' and str(col) not in ['da_membership', 'language', 'city', 'region']:
        counts = Counter()
        users[col].dropna().apply(lambda x: counts.update(x))
        sorted_counts = counts.most_common()
        totals[col] = sorted_counts

for i, col in enumerate(totals):
    display(f"Column: {col}", totals[col])

'Column: da_membership'

Active     1297
Current    1103
            275
Name: da_membership, dtype: Int64

'Column: language'

English    5126
Spanish    1303
French     1083
German      800
Dutch       569
           ... 
Xhosa         1
Breton        1
Venda         1
Sindhi        1
Sesotho       1
Name: language, Length: 111, dtype: int64

'Column: city'

...

[]

'Column: contributions'

[('modules', 13609),
 ('patches', 13277),
 ('services', 13217),
 ('issues', 11187),
 ('forums', 8387),
 ('themes', 6426),
 ('documentation', 6186),
 ('translations', 4980),
 ('irc', 4292),
 ('drupalorg', 3711),
 ('profiles', 3430),
 ('mentor', 3033),
 ('applications', 2810),
 ('tests', 2048),
 ('drupalcon', 1119),
 ('slack', 1069),
 ('stackexchange', 1015),
 ('code-of-conduct', 42),
 ('discover-drupal-mentor', 35),
 ('drupalcon-north-america-2021-individual-sponsor', 15),
 ('discover-drupal-graduate', 9)]

'Column: events'

[('chicago_2011', 2293),
 ('denver_2012', 2061),
 ('sf_2010', 2061),
 ('portland_2013', 2023),
 ('austin_2014', 1813),
 ('amsterdam_2014', 1707),
 ('losangeles_2015', 1551),
 ('munich_2012', 1540),
 ('london_2011', 1475),
 ('barcelona_2015', 1463),
 ('dc_2009', 1447),
 ('prague_2013', 1427),
 ('neworleans_2016', 1427),
 ('baltimore_2017', 1297),
 ('paris_2009', 1162),
 ('nashville_2018', 1147),
 ('copenhagen_2010', 1092),
 ('boston_2008', 1055),
 ('dublin_2016', 1048),
 ('seattle_2019', 1041),
 ('vienna_2017', 952),
 ('amsterdam_2019', 868),
 ('barcelona_2007', 803),
 ('szeged_hungary_2008', 778),
 ('asia_2016', 743),
 ('global_2020', 721),
 ('vancouver_2006', 670),
 ('antwerp_2005', 659),
 ('oscms_2007', 622),
 ('london_2005', 621),
 ('portland_2005', 612),
 ('minneapolis_2020', 610),
 ('brussels_2006', 608),
 ('northamerica_2021', 574),
 ('lille_2023', 571),
 ('europe_2018', 569),
 ('amsterdam_2005', 568),
 ('prague_2022', 524),
 ('pittsburgh_2023', 471),
 ('portland_2022', 411),
 ('

### Cleaning

There is a lot of empty values in this dataset. Let's try to clean it!

In [54]:
# Identify SPAM users.
# There are certainly **not** 2 million active users on *d.o*.

### Users analysis

...

#### Assumptions

#### Questions

In [55]:
# User registration.
first_user = users.sort_values('registered_on').iloc[0]
display(f"First user registered: ({first_user.id}) {first_user.title} registered on: {first_user.registered_on}")

# Number of users registered on each day.
# Number of users registered on each month.
# Number of users registered on each year.
# Number of users registered on each year.

'First user registered: (1) dries registered on: 2001-03-31 13:43:00'

## Geographical distribution

Exploration of the distribution of the Drupal community on the planet.

In [ ]:
# Plot region in bar
# Scatter cities on a map?

## Expertises

Analyze **expertises** of the users community.

* Are certain professional sectors over-represented?
* Are there regions of the world where certain areas of expertise are favored?

In [ ]:
# Group by `field_industries_worked_in` and histplot user counts
# Segment by `field_country` 

## Mentors and mentees

Exploration of the mentorship within the Drupal community.